In [1]:
import pickle
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import date
from collections import namedtuple

In [2]:
import cafle as cf
from cafle.genfunc import rounding as R
from cafle.genfunc import percent as P

In [3]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"),
      pd.get_option("display.max_rows"))

40 200


# Initial Setting

In [4]:
# ctg = namedtuple('category', ['nml', 'cld'])

In [5]:
def ctg(nml, cld):
    idx = ['nml', 'cld']
    return Series([nml, cld], index = idx)

In [6]:
# Pickling Data
with open("data/area.pickle", "rb") as fr:
    data_area = pickle.load(fr)
areadf = data_area['areadf']

In [7]:
# Call out the area data
area_rent = areadf.loc[[x in data_area['rent_col'] for x in areadf.name]]
area_rent = area_rent.area_py.groupby(areadf['floor']).sum()
area_cld = round(area_rent[['b1', 'f1']].sum(), 2)
area_nml = round(area_rent[['f2', 'f3', 'f4']].sum(), 2)

# Valuation
### 1) Assumption

In [8]:
class Valuation:
    def __init__(self, rent, mtnc, cap_rate, area, vcncy_rate, oprtg_rate, IR, RU=1_000_000):
        self.rent = rent
        self.mtnc = mtnc
        self.cap_rate = cap_rate
        self.area = area
        self.vcncy_rate = vcncy_rate
        self.oprtg_rate = oprtg_rate
        self.IR = IR
        self.RU = RU
        
        self.__intlz__()
    
    def __intlz__(self):
        self.dpst_amt = self.rent * 10 * self.area / self.RU # deposit amount
        self.rent_amt = self.rent * 12 * self.area / self.RU # yearly rent amount
        self.mtnc_amt = self.mtnc * 12 * self.area / self.RU # yearly maintenance cost
        
        self.PGI = (self.dpst_amt * self.IR) + self.rent_amt + self.mtnc_amt
        self.EGI = self.PGI * (1 - self.vcncy_rate)
        self.oprtg_cst = self.PGI * self.oprtg_rate
        self.NOI = self.EGI - self.oprtg_cst
        self.value = self.NOI / self.cap_rate
        self.value_sum = self.value.sum()

In [57]:
class Val_Mtrx:
    def __init__(self, rent=None, mtnc=None, cap_rate=None, area=None, 
                 vcncy_rate=None, oprtg_rate=None, IR=None, RU=1_000_000):
        self.rent = rent
        self.mtnc = mtnc
        self.cap_rate=cap_rate
        self.area = area
        self.vcncy_rate = vcncy_rate
        self.oprtg_rate = oprtg_rate
        self.IR = IR
        self.RU = RU
        
    def value(self, rent=None, mtnc=None, cap_rate=None, area=None, 
                 vcncy_rate=None, oprtg_rate=None, IR=None):
        keys = ['rent', 'mtnc', 'cap_rate', 'area', 'vcncy_rate', 'oprtg_rate', 'IR']
        dct = {}
        for key in keys:
            if locals()[key] is not None:
                tmp_val = locals()[key]
            else:
                tmp_val = getattr(self, key)
            dct['tmp_'+key] = tmp_val
            
        tmpistnc = Valuation(rent = dct['tmp_rent'],
                             mtnc = dct['tmp_mtnc'],
                             cap_rate = dct['tmp_cap_rate'],
                             area = dct['tmp_area'],
                             vcncy_rate = dct['tmp_vcncy_rate'],
                             oprtg_rate = dct['tmp_oprtg_rate'],
                             IR = dct['tmp_IR'])
        return tmpistnc
    

In [58]:
val_mtrx = Val_Mtrx(mtnc = ctg(2_000, 2_000),
                    area = ctg(area_nml, area_cld),
                    vcncy_rate = ctg(0.0, 0.0),
                    oprtg_rate = P(14.5),
                    IR = P(1.0))

In [59]:
val = val_mtrx.value(rent = ctg(20_000, 44_000),
                     cap_rate = ctg(P(4.5), P(5.0)))

In [38]:
tmp_mtnc

NameError: name 'tmp_mtnc' is not defined

In [245]:
k = abc.value(rent=ctg(20_000, 44_000))
k.__dict__

{'rent': nml    20000
 cld    44000
 dtype: int64,
 'mtnc': nml    2000
 cld    2000
 dtype: int64,
 'cap_rate': nml    0.045
 cld    0.049
 dtype: float64,
 'area': nml    8081.72
 cld    6619.06
 dtype: float64,
 'vcncy_rate': nml    0.0
 cld    0.0
 dtype: float64,
 'oprtg_rate': 0.145,
 'IR': 0.01,
 'RU': 1000000,
 'dpst_amt': nml    1616.3440
 cld    2912.3864
 dtype: float64,
 'rent_amt': nml    1939.61280
 cld    3494.86368
 dtype: float64,
 'mtnc_amt': nml    193.96128
 cld    158.85744
 dtype: float64,
 'PGI': nml    2149.737520
 cld    3682.844984
 dtype: float64,
 'EGI': nml    2149.737520
 cld    3682.844984
 dtype: float64,
 'oprtg_cst': nml    311.711940
 cld    534.012523
 dtype: float64,
 'NOI': nml    1838.025580
 cld    3148.832461
 dtype: float64,
 'value': nml    40845.012880
 cld    64261.886966
 dtype: float64,
 'value_sum': 105106.89984571429}

In [167]:
def value(rent_val, cap_rate):
    tmpistnc = Valuation(rent = rent_val,
                       mtnc = ctg(2_000, 2_000),
                       cap_rate = cap_rate,
                       area = ctg(area_nml, area_cld),
                       vcncy_rate = ctg(0.0, 0.0),
                       oprtg_rate = P(14.5),
                       IR = P(1.0))
    return tmpistnc

def value_to_df(value:Valuation): # <- valuation instance를 인자로 받음
    tmp_dct = {}
    for key in val_keys:
        tmp_dct[key] = getattr(value, key)
    return DataFrame(tmp_dct)

In [158]:
def value_dict(rent_val:dict, cap_rate:dict):
    tmpdict = {}
    for case_key, case_val in rent_case.items():
        for cap_key, cap_val in cap_rate.items():
            tmpdict[case_key + "_" + cap_key] = value(case_val, cap_val)
    return tmpdict

In [138]:
rent_case = {'case1': ctg(20_000, 44_000), # unit rent
             'case2': ctg(21_000, 46_000),
             'case3': ctg(22_000, 48_000),
             'case4': ctg(23_000, 50_000)}
cap_rate = {'cap1': ctg(P(4.3), P(4.8)),
            'cap2': ctg(P(4.4), P(4.9)),
            'cap3': ctg(P(4.5), P(5.0))}
val_dict = value_dict(rent_case, cap_rate)

In [159]:
val_keys = ['rent', 'mtnc', 'cap_rate', 'area','vcncy_rate', 'dpst_amt', 
           'rent_amt', 'mtnc_amt', 'PGI', 'EGI', 'oprtg_cst', 'NOI', 'value']


In [152]:
val_dict['case1_cap1'].oprtg_cst

nml    311.711940
cld    534.012523
dtype: float64

In [147]:
DataFrame({'rent':getattr(val_dict['case1_cap1'], 'rent'), 
 'mtnc':getattr(val_dict['case1_cap1'], 'mtnc')})

,rent,mtnc
nml,20000,2000
cld,44000,2000


In [128]:
rent_case = {'case1': ctg(20_000, 44_000), # unit rent
        'case2': ctg(21_000, 46_000),
        'case3': ctg(22_000, 48_000),
        'case4': ctg(23_000, 50_000)}
cap_rate = [P(4.5), P(4.7), P(4.9), P(5.1)]
tmpdict = {}
for case in rent_case:
    for cap in cap_rate:
        tmpdict[case + "_" + str(cap*100)] = value(ctg(23_000, 50_000), cap)
val_dict = tmpdict

In [130]:
val_dict['case1_4.5'].__dict__.keys()

dict_keys(['rent', 'mtnc', 'cap_rate', 'area', 'vcncy_rate', 'oprtg_rate', 'IR', 'RU', 'dpst_amt', 'rent_amt', 'mtnc_amt', 'PGI', 'EGI', 'oprtg_cst', 'NOI', 'value', 'value_sum'])

In [133]:
val_dict['case1_4.5'].NOI / ctg(P(4.3), P(5.0))

nml    48577.997265
cld    71193.947454
dtype: float64

In [134]:
val_dict['case1_4.5'].NOI

nml    2088.853882
cld    3559.697373
dtype: float64

In [84]:
case1 = Valuation(rent = ctg(23_000, 50_000),
                  mtnc = ctg(2_000, 2_000),
                  cap_rate = P(4.5),
                  area = ctg(area_nml, area_cld),
                  vcncy_rate = ctg(0.0, 0.0),
                  oprtg_rate = P(14.5),
                  IR = P(1.0))

case1.value_sum

125523.36122400002

In [46]:
RU = 1_000_000 # rounding unit
rent = ctg(23_000, 50_000) # unit rent
mtnc = ctg(2_000, 2_000) # unit maintenance cost
cap_rate = [P(4.5), P(4.7), P(4.9), P(5.1)]
area = ctg(area_nml, area_cld) # rent area
vcncy_rate = ctg(0.0, 0.0) # vacancy rate
oprtg_rate = P(14.5) # operating rate
IR = P(1.0)

In [47]:
dpst_amt = rent * 10 * area / RU # deposit amount
rent_amt = rent * 12 * area / RU # yearly rent amount
mtnc_amt = mtnc * 12 * area / RU # yearly maintenance cost

In [59]:
PGI = (dpst_amt * IR) + rent_amt + mtnc_amt
EGI = PGI * (1 - vcncy_rate)
oprtg_cst = PGI * oprtg_rate
NOI = EGI - oprtg_cst
value = DataFrame({x : NOI / x for x in cap_rate})
value_sum = DataFrame([value.sum()], index=['sum'])
value_prt = value.append(value_sum)

In [61]:
value_prt

,0.045,0.047,0.049,0.051
nml,46418.975164,44443.699625,42629.671069,40957.919262
cld,79104.386060,75738.241972,72646.885157,69797.987700
sum,125523.361224,120181.941597,115276.556226,110755.906962


In [71]:
RU = 1_000_000 # rounding unit
rent = {'case1': ctg(20_000, 44_000), # unit rent
        'case2': ctg(21_000, 46_000),
        'case3': ctg(22_000, 48_000),
        'case4': ctg(23_000, 50_000)}
mtnc = ctg(2_000, 2_000) # unit maintenance cost
cap_rate = [P(4.5), P(4.7), P(4.9), P(5.1)]
area = ctg(area_nml, area_cld) # rent area
vcncy_rate = ctg(0.0, 0.0) # vacancy rate
oprtg_rate = P(14.5) # operating rate
IR = P(1.0)

In [ ]:
value_data = DataFrame(columns=['case', 'rent_nml', 'rent_cld', 
                                'mtnc_nml', 'mtnc_cld', 'cap_rate', 
                                'val_nml', 'val_cld', 'value'])
for key, rent_item in rent.items():
    tmp = {'case': key,
           'rent_nml': rent_item.nml,
           'rent_cld': rent_item.cld,
           'mtnc_nml': mtnc.nml,
           'mtnc_cld': mtnc.cld,
           'cap_rate': cap_rate}

In [73]:
value = DataFrame()
for key, item in rent.items():
    dpst_amt = item * 10 * area / RU # deposit amount
    rent_amt = item * 12 * area / RU # yearly rent amount
    mtnc_amt = mtnc * 12 * area / RU # yearly maintenance cost

    PGI = (dpst_amt * IR) + rent_amt + mtnc_amt
    EGI = PGI * (1 - vcncy_rate)
    oprtg_cst = PGI * oprtg_rate
    NOI = EGI - oprtg_cst
    tmpdf = DataFrame({x : NOI / x for x in cap_rate})
    value = value.append(tmpdf)
    
    
    
#value_sum = DataFrame([value.sum()], index=['sum'])
#value_prt = value.append(value_sum)

In [74]:
value

,0.045,0.047,0.049,0.051
nml,40845.012880,39106.927226,37510.726114,36039.717247
cld,69974.054696,66996.435347,64261.886966,61741.812967
nml,42703.000308,40885.851359,39217.041099,37679.117919
cld,73017.498484,69910.370889,67056.886363,64427.204545
nml,44560.987736,42664.775492,40923.356084,39318.518591
cld,76060.942272,72824.306431,69851.885760,67112.596122
nml,46418.975164,44443.699625,42629.671069,40957.919262
cld,79104.386060,75738.241972,72646.885157,69797.987700


In [72]:
rent.items()

dict_items([('case1', nml    20000
cld    44000
dtype: int64), ('case2', nml    21000
cld    46000
dtype: int64), ('case3', nml    22000
cld    48000
dtype: int64), ('case4', nml    23000
cld    50000
dtype: int64)])